**目次**<a id='toc0_'></a>    
- 1. [科学技術計算2課題](#toc1_)    
  - 1.1. [課題02-1：float16の丸め誤差](#toc1_1_)    
  - 1.2. [課題02-2：浮動小数点数演算](#toc1_2_)    
  - 1.3. [課題02-3：絶対誤差と相対誤差を組み合わせた収束判定](#toc1_3_)    
  - 1.4. [課題02-4：binary64ビット列からfloat64への復元](#toc1_4_)    
  - 1.5. [課題02-5：softmaxの実装と数値安定性の比較](#toc1_5_)    
  - 1.6. [課題02-6：マシンイプシロンの計算](#toc1_6_)    
  - 1.7. [課題02-7：2次方程式と桁落ちの回避](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[科学技術計算2課題](#toc0_)


## 1.1. <a id='toc1_1_'></a>[課題02-1：float16の丸め誤差](#toc0_)

演習資料では，IEEE 754 binary64 を用いた場合に，次のような現象をビット単位で説明した．

- `0.1 + 0.2` と `0.3` は一致しないこと
- `(0.1 + 0.2) - 0.3` は `0` ではないこと

この課題では，IEEE 754 binary16（`np.float16`）を用いて同様の検証を行う．

**やること**

1. `0.1`, `0.2`, `0.3` を `float16` に変換し，それぞれの **ビット表現** を示す．
2. `0.1 + 0.2` の計算を，演習資料と同様の手順で（ただし`float16`で）ビット単位で説明する．
   - 加算前の整列（指数の調整）
   - 加算処理
   - 正規化
   - 丸め（0捨1入）
3. `(0.1 + 0.2) - 0.3` の計算についても同様に手順を示す．
4. 実際に `np.float16` を用いた Python 実装結果と比較し，一致を確認する．

**注意点**

- IEEE 754 binary16 のフォーマット（符号ビット1，指数5，仮数10，バイアス15）は各自調べること．


## 1.2. <a id='toc1_2_'></a>[課題02-2：浮動小数点数演算における桁落ち](#toc0_)

次のコードの出力は本来 18.0 であるはずだが，実際には 32.0 が出力される．
この理由を，浮動小数点数の **演算手順をビット単位で追跡する**ことで説明せよ．

```python
a = 18.0
b = 2.0**57
c = (a + b) - b
print(c)
```


**やること**



**注意点**
- ビット列は長さ64の文字列（0と1）として扱い，符号・指数・仮数の各部分を明示的に分けて説明すること．
- 丸め誤差がどのように結果へ影響したかを確認すること．


## 1.3. <a id='toc1_3_'></a>[課題02-3：絶対誤差と相対誤差を組み合わせた収束判定](#toc0_)

円周率を計算するライプニッツの公式
$$
S_n = 4 \sum_{i=0}^n \frac{(-1)^i}{2i+1}
$$
の収束判定を行う．

**やること**

1. $n = 0, 1, \dots, n_{\max}$ について $S_n$ を計算するコードを実装する．
2. 「$S_n$ と $S_{n-1}$ の絶対誤差 $| S_n - S_{n-1}|$ がしきい値 $0.1$ 以下」という条件で反復を終了する単純な終了条件を実装する．
3. 絶対誤差だけでなく，相対誤差を組み合わせた終了条件を検討・考案し，実装する．
4. 改善前後で，反復回数や近似値の精度を比較する．
   - 実際の $\pi$ の値（`np.pi`）と比較し，近似精度を確認する．


**注意点**

- しきい値を適切に設定すること（マシンイプシロン以下にはしない）
- この公式は収束が非常に遅いため，しきい値を小さくしすぎると反復数が膨大になる．
- 絶対誤差だけでなく相対誤差も併用し，より汎用的な収束判定を考えること．


## 1.4. <a id='toc1_4_'></a>[課題02-4：binary64ビット列からfloat64への復元](#toc0_)


In [ ]:
import numpy as np
from typing import Tuple


def float64_to_binary(
    f: float | np.float64,
    separate: bool = False
) -> str | Tuple[str, str, str]:
    """convert float64 to 64 bits as string

        Args:
        f (float | np.float64): IEEE754 binary64 floating point number
        separate (bool, optional): return tuple of (sign bit, exponent bits, mantissa bits) if True,
            return 64 bits if False. Defaults to False.

    Returns:
        str or Tuple of str: bit string of f
    """
    assert isinstance(f, float)
    from struct import pack, unpack

    # see https://note.nkmk.me/python-float-hex/
    s = format(unpack('>Q', pack('>d', f))[0], "064b")

    if separate:
        return s[0:1], s[1:12], s[12:]
    else:
        return s


演習資料では，float64を受け取り，
IEEE754 binary64を表す64ビットを長さ64の文字列を返す下記の関数
`float64_to_binary`
を説明した．この関数の挙動は以下の通りである．


In [ ]:
binary_string = float64_to_binary(0.1)
binary_string

詳しく説明はしないが，
この関数内部では，以下のpythonの関数pack/unpackを用いて変数値のバイト列を取得し，文字列に変換している．

- https://docs.python.org/ja/3/library/struct.html#struct.pack
- https://docs.python.org/ja/3/library/struct.html#struct.unpack

したがってpack/unpackを用いてこれの逆を行えば，文字列から変数値に変換することはできる（説明はしない）．

In [ ]:
import struct
struct.unpack('>d', struct.pack('>Q', int(binary_string, 2)))[0]


この課題では，上記のようなpack/unpackなど用いず，定義通りに実装する．
つまり，IEEE754 binary64を表す64ビットを長さ64の文字列として受け取り，
float64として返す関数を作成する．


**やること**

1. IEEE 754 binary64 の 64 ビット列（長さ 64 の文字列）を入力として受け取り，`np.float64` を返す関数 `binary_to_float64` を**定義通り**に実装する（`pack/unpack` や構造体などは用いない）．
2. 正規化数・非正規化数・`nan`・`inf`・`-inf`・`0`・`-0` に**すべて対応**する．
3. 内部処理では次の手順を厳密に実装する．
   - Step1：**符号ビット**（最上位ビット）により符号を決定する．
   - Step2：**仮数部**（下位 52 ビット）を2進小数として10進実数に変換する．
     - 正規化数と非正規仮数で仮数部の先頭ビットの解釈を変更する．
   - Step3：**指数部**（中間 11 ビット）をバイアス（1023）を考慮して2の冪を計算する．

**関数定義**

```python
def binary_to_float64(
    bit: str,
) -> np.float64:
```

**注意事項**

- 演習資料ノートブックのコードや関数を利用・改変して作成すること．

**実行手順**

1. 正規化数（一般的な値，最大・最小の値），非正規化数（一般的な値，最大・最小の値），`nan`, `inf`, `-inf`, `0`, `-0`といったfloat64の数値を，まず`float64_to_binary()`を使って長さ64の文字列に変換する．
2. その 64 ビット文字列を，作成した`binary_to_float64()`に入力し，得られた数値が元のfloat64の数値と一致していることを確認する．
   - 正規化数や非正規化数，`0`，`-0`などについては，等価演算子`==`を用いて比較する．
       - ビット列として完全に一致していることを確認するため，`isclose()`関数などは使用せず，直接比較を行う．
   - `nan`，`inf`，`-inf`などについては，`np.isnan()`や`np.isinf()`を使用して比較する．

## 1.5. <a id='toc1_5_'></a>[課題02-5：softmaxの実装と数値安定性の比較](#toc0_)

演習資料では，関数`exp`がオーバーフローしやすいことを説明した．
そのため，数式通りに愚直に`softmax`を実装すると，
すぐにオーバーフローを起こし，数値的に安定ではない．

この課題では，以下の2つの`softmax`関数を実装する．

**やること**

1. 以下の2種類の `softmax` 関数を実装する．
    - 愚直に実装した数値的に不安定な softmax
    - 演習資料で説明した，数値的に安定な softmax
2. 実装した関数の出力を，`scipy.special.softmax` の結果と比較し，安定性を確認する．

**関数定義**

```python
def softmax(
    x: np.ndarray,
) -> np.ndarray:
```

**実行手順**

1. すべての値は **e表記** を用いて，小数第15位まで表示する．
   - 例：`print(f"{value:.15e}")`
1. 愚直な実装ではオーバーフローするような値の範囲で，ランダムに多数の`np.float64`型の`ndarray`を生成する．
2. 実装した2つの`softmax`関数と，`scipy.special.softmax`の結果を比較する．
   - 数値的に安定な実装では，`scipy.special.softmax`と一致することを確認する．


## 1.6. <a id='toc1_6_'></a>[課題02-6：マシンイプシロンの計算](#toc0_)


$1 + \epsilon > 1$となる（つまり演算の前後で値が変化する）
ような最小の浮動小数点数$\epsilon$をマシンイプシロンと呼ぶ．
この課題ではシンイプシロンの値を確かめる．

**やること**

次のアルゴリズムを用いて，マシンイプシロンを計算する．

1. $\epsilon \leftarrow 1$
2. $1 + \epsilon > 1$ならばStep3へ，そうでないならStep4へ
3. $\epsilon \leftarrow \epsilon / 2$として，値を表示してからStep2へ
4. $\epsilon \leftarrow 2 \epsilon$をマシンイプシロンとして表示して終了


**実行手順**

1. すべての値は **e表記** を用いて，小数第15位まで表示する．
   - 例：`print(f"{value:.15e}")`

2. マシンイプシロンの計算を，以下の3つの型で行う．
   - 64ビット浮動小数点型：`np.float64`
   - 32ビット浮動小数点型：`np.float32`
   - 16ビット浮動小数点型：`np.float16`

3. 計算した結果が`np.finfo()`で取得できるマシンイプシロンと一致していることを確認する．

4. 計算した結果が型の定義から手計算できるマシンイプシロンと一致していることを確認する．



## 1.7. <a id='toc1_7_'></a>[課題02-7：2次方程式と桁落ちの回避](#toc0_)

次の2次方程式を考える．
$$
     f(x) = x^2 - 2ax + b = 0, \quad a > 0, \quad D = a^2 - b > 0
$$
このとき2つの解は解の公式から
$$
x_1 = a + \sqrt{D}, x_2 = a - \sqrt{D}
$$
である．
この計算において，$|b|$ が $a^2$ に比べて小さいとき，解の公式をそのまま用いると **桁落ち** が発生する．


**やること**

1. 以下の2つの方法で解を計算するコードを実装する．
    - 解の公式による計算（桁落ちの可能性がある）
    - 改善した計算式（桁落ちを回避する式を導出すること）
2. $b$の値を1に固定して，$a$の値を$1, 10^{1}, 10^{2}, \ldots$と大きくしていき，それぞれの方法で桁落ちが発生するかどうかを確認する．
   - $b=1$のとき，$\lim_{a \to \infty} x_2 = 0$であるが，$a$が有限であれば$x_2 \neq 0$である．
   - しかし桁落ちが発生すると，$a$が有限でも$x_2 = 0$となってしまう．


**実行手順**

1. すべての値は **e表記** を用いて，小数第15位まで表示する．
   - 例：`print(f"{value:.15e}")`

1. 横軸に$a$（対数目盛），縦軸に2つの方法で計算した$x_2$（対数目盛）をプロットする．$a$がどのくらい大きくなると桁落ちが発生するかを確認する．

2. 浮動小数点のビット数を変えて同様の検証を行う．（$a$の最大値はそれぞれの型に合わせること）
   - 64ビット浮動小数点型：`np.float64`
   - 32ビット浮動小数点型：`np.float32`
   - 16ビット浮動小数点型：`np.float16`

注意：各型で暗黙の型変換が行われないように，計算対象の変数について明示的に型を指定し，`.dtype` などで確認すること．（何も指定しない場合には，pythonとnumpyはデフォルトでfloat64を用いてしまう）
